In [243]:
import collections
import vk
import pandas as pd
import time
from vk.exceptions import VkAPIError

In [244]:
VK_TOKEN = 'f052c5646f85109471fc9ae1fd2b25a9b8f10f86d83c254e9efbc0f453897f1e251c4a4304e8933bec20d'
ID = 10385605
session = vk.Session(VK_TOKEN)
vk_api = vk.API(session, v=5.103)
deq = collections.deque(maxlen=4)

In [245]:
def jaccard(l1, l2):
    intersec = len(l1.intersection(l2))
    union = len(l1.union(l2))
    return intersec / union

In [246]:
def median(l):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    l = sorted(l)
    if len(l) % 2 == 1:
        return l[int(len(l) / 2)]
    else:
        return (l[int(len(l) / 2 - 1)] + l[int(len(l) / 2)]) / 2

In [247]:
def get_med(id):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    count_friends = []
    groups = vk_api.groups.get(user_id=id)
    for group in groups['items']:
        try:
            count_friends.append(vk_api.groups.getMembers(group_id=group, filter='friends')['count'])
        except Exception:
            continue
    return median(count_friends)

In [248]:
def get_groups_activity(id):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    groups = vk_api.groups.get(user_id=id, fields=['activity'], extended=1)
    activities = []
    for group in groups['items']:
        try:
            activities.append(group['activity'])
        except Exception:
            continue
    return activities

In [249]:
def get_percents(activities):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    df = pd.DataFrame(data= activities, columns=['activity'])
    df = df.loc[(df.loc[:, 'activity'] != 'Открытая группа') & (df.loc[:, 'activity'] != 'Закрытая группа')]
    df['count'] = 1
    df2 = df.groupby('activity').count()
    df2['percent'] = df2['count'] / len(df.index) * 100
    df2 = df2.sort_values(by = 'percent', ascending = False)
    activities = df2.index
    df2['activity'] = activities
    df2['index'] = range(len(df2))
    df2 = df2.set_index('index')
    return df2.head(10)

In [250]:
def get_my_lieber_freund(first_p, second_p):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    count = 0
    for i in range(len(first_p)):  
        for j in range(len(second_p)):
            if (first_p.loc[i, 'activity'] == second_p.loc[j, 'activity']) &  (abs(first_p.loc[i, 'percent'] - second_p.loc[j, 'percent']) < 5):
                count += 1
    return count

In [251]:
my_activities = get_groups_activity(ID)
my_percents = get_percents(my_activities)
friends = vk_api.friends.get(user_id=ID, extended=1, fields='nickname')['items']
my_activities = get_groups_activity(ID)
df = pd.DataFrame(columns=['id', 'jaccard', 'median', 'percent'])
i = 0
file = open('rating.csv', 'w')
file.write('index,id,similarity,median,count\n')
file.close()

In [252]:
def get_sumba():
    global i
    global df
    global friends
    global file
    for friend in friends:
        try:
            print(len(friends))
            print(" ")
            print('friend #{}, id: {} '.format(i, friend['id']))
            if (friend.get('deactivated') != None):
                print('remove deactivated account: {}'.format(friend['id']))
                friends.remove(friend)
                print('removed diactivated account!')
            el if (friend['is_closed'] == 'True'):
                print('remove private account: {}'.format(friend['id']))
                friends.remove(friend)
                print('removed private account!')
            elif (vk_api.groups.get(user_id=friend['id'])['count'] == 0):
                print('remove account without groups! id: {}'.format(friend['id']))
                friends.remove(friend)
                print('removed friend')                
            else:
                activities = get_groups_activity(friend['id'])
                similarity = jaccard(set(my_activities), set(activities))
                print('get jaccard {} for friend: {}'.format(i, friend['id']))

                med = get_med(friend['id'])
                print('get med {} for friend: {}'.format(i, friend['id']))

                fr_percents = get_percents(activities)
                count = get_my_lieber_freund(my_percents, fr_percents)
                print('get count {} for friend: {}'.format(i, friend['id']))
                file = open('rating.csv', 'a+')
                file.write('{},{},{},{},{}\n'.format(i, friend['id'], similarity, med, count))
                file.close()
                friends.remove(friend)
                i += 1
                print(" ")
#             df.loc[i] = [int (friend['id']), similarity, int (med), int (count)]
#             print(df.loc[i])
        except VkAPIError as e:
            if e.code == 18:
#                 or VkAPIError.code == 30 or VkAPIError.code == 113 or VkAPIError.code == 203:
                friends.remove(friend)
                print('account #{} is closed or deactivated!'.format(friend['id']))
            time.sleep(1)
            continue

In [253]:
def get_rating():
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    while(len(friends) != 0):
        get_sumba()

In [255]:
get_rating()

373
 
friend #13, id: 506116 
get jaccard 13 for friend: 506116
get med 13 for friend: 506116
get count 13 for friend: 506116
 
372
 
friend #14, id: 2502768 
372
 
friend #14, id: 3152352 
remove private/deactivated account: 3152352
371
 
friend #14, id: 3817808 
get jaccard 14 for friend: 3817808
get med 14 for friend: 3817808
get count 14 for friend: 3817808
 
370
 
friend #15, id: 6691754 
370
 
friend #15, id: 8794146 
370
 
friend #15, id: 9679112 
370
 
friend #15, id: 10391867 
370
 
friend #15, id: 10950395 
get jaccard 15 for friend: 10950395
get med 15 for friend: 10950395
get count 15 for friend: 10950395
 
369
 
friend #16, id: 11750565 
get jaccard 16 for friend: 11750565
369
 
friend #16, id: 12427044 
369
 
friend #16, id: 13293707 
get jaccard 16 for friend: 13293707
get med 16 for friend: 13293707
get count 16 for friend: 13293707
 
368
 
friend #17, id: 20522572 
368
 
friend #17, id: 21040208 
368
 
friend #17, id: 21923765 
get jaccard 17 for friend: 21923765
get m

ReadTimeout: HTTPSConnectionPool(host='api.vk.com', port=443): Read timed out. (read timeout=10)

In [193]:
diactive = 3152352
closed_group = 40674322

In [260]:
t = vk_api.groups.get(user_id=116796010)['count']
t

VkAPIError: 30. This profile is private. request_params = {'method': 'groups.get', 'oauth': '1', 'v': '5.103', 'user_id': '116796010'}

In [261]:
len(friends)

54

In [267]:
for fr in friends:
    if (fr.get('deactivated') == None):
        friends.remove(fr)
    elif (fr.get('is_closed') == None):
        print('DIACTIVATED')
    print(' ')

print(len(friends))

True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
True
 
27


In [ ]:
# сохранять индекс друга, после завершение цикла удалять и снова подавать на вход